In [14]:
import json
import os
import requests
from datetime import datetime

# --- 1. Your Market IDs ---
IDs = [
    "8943772447983355726067120991407169458512217556617393908885683412829797811759",
    "2220541104128638095447408349581828110649734278725441453904057326328035654364",
    "62595435619678438799673612599999067112702849851098967060818869994133628780778",
    "20403398307334234131402204643675486921930048779742580789086247611446319828926",
    "15704223254061637699665008299235004894744524204070683496068480275285392327272",
    "8249069320464381596140964768863147701990164952141635813146919319175257044012",
    "51171279104285400909666677762285621845926753560865042481170667672124752488918",
    "50692033443703929669850515798357627200251915622504785351531528038278368575099",
    "33156410999665902694791064431724433042010245771106314074312009703157423879038",
    "90883464094985571010902349297672923777772590459152587087503246655308316600180",
    "80339409420962166114161973004452881729789538477338621544329890236335726311527",
    "168386216037015250049091014752451659613521676425580827909510879873159107969",
    "13294549611854156060952327202052855585194847446707854951585060180328600525494",
    "49240924317962054148054803135220188899268212906425611341195210349274953582528",
    "79198300868830485284678716507005480871569275475252085914785144227204298869058",
    "111376742935340636514883997859794030073097390424337351101383349939644307757449",
    "101388243441004569906471025818583855737097216415383057042720138776689258624402",
    "83881708902836152136079089518249178414226067834465945166459477633554187275598",
    "113656537263060579808707960104112980070257881368251651103052234993098850420305"
]

# --- 2. Helper Functions ---

def format_date(date_str):
    """Safely formats ISO date strings to YYYY-MM-DD HH:MM UTC."""
    if not date_str: 
        return None
    try:
        # Standardize ISO format
        dt = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
        return dt.strftime('%Y-%m-%d %H:%M UTC')
    except (ValueError, TypeError):
        return date_str

def fetch_and_save_market_data(ID):
    """Fetches market data from Polymarket API and saves to JSON."""
    url = "https://gamma-api.polymarket.com/markets"
    output_filename = f"{ID}_polymarket_data.json"
    
    # Skip if already exists to save bandwidth/time
    if os.path.exists(output_filename):
        return output_filename

    params = {"clob_token_ids": ID}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        return output_filename
        
    except Exception as e:
        print(f"Error fetching {ID}: {e}")
        return None

def extract_universal_polymarket_info(json_input):
    """Universally parses any Polymarket JSON (Sports, Politics, Crypto, etc.)."""
    
    # Load Data
    if isinstance(json_input, str):
        if os.path.isfile(json_input):
            with open(json_input, 'r', encoding='utf-8') as f:
                data = json.load(f)
        else:
            data = json.loads(json_input)
    else:
        data = json_input

    # Get Market Object
    if isinstance(data, list) and len(data) > 0:
        market = data[0]
    elif isinstance(data, dict):
        market = data
    else:
        return {"Error": "Invalid JSON structure"}

    # Extract Logic
    event = market.get('events', [{}])[0]
    series = event.get('series', [{}])[0]

    # Parse Outcomes (Handle strings vs lists)
    try:
        raw_outcomes = market.get('outcomes')
        raw_prices = market.get('outcomePrices')
        
        outcomes = json.loads(raw_outcomes) if isinstance(raw_outcomes, str) else (raw_outcomes or ["Unknown"])
        prices = json.loads(raw_prices) if isinstance(raw_prices, str) else (raw_prices or [])
    except (json.JSONDecodeError, TypeError):
        outcomes, prices = ["Error Parsing"], []

    # Build Data Dictionary
    info = {
        "Market Core": {
            "Question": market.get('question'),
            "Status": "Closed" if market.get('closed') else "Active",
            "Result": "Resolved" if market.get('umaResolutionStatus') == "resolved" else "Pending"
        },
        "Financials": {
            "Volume ($)": float(market.get('volume', 0)),
            "Liquidity": market.get('liquidity', "0"),
        },
        "Timestamps": {
            "End Date": format_date(market.get('endDate')),
            "Event Start": format_date(market.get('gameStartTime') or market.get('startDate')),
        },
        "Context": {
            "Category": series.get('title', 'Uncategorized'), 
            "Group/Tag": market.get('groupItemTitle', 'Single Market'),
        },
        "Outcomes": {
            outcome: float(price) for outcome, price in zip(outcomes, prices)
        }
    }

    # Conditional Additions
    if 'score' in event and event['score']:
        info["Context"]["Score"] = event['score']

    return info

# --- 3. Main Execution Loop ---

if __name__ == "__main__":
    print(f"Processing {len(IDs)} markets...\n")
    print("="*60)

    for ID in IDs:
        # 1. Fetch File
        filename = fetch_and_save_market_data(ID)
        
        if filename:
            # 2. Extract Info
            data = extract_universal_polymarket_info(filename)
            
            # 3. Print Readable Report
            if "Error" in data:
                print(f"Error processing {ID}: {data['Error']}")
            else:
                q = data['Market Core']['Question']
                cat = data['Context']['Category']
                vol = data['Financials']['Volume ($)']
                end = data['Timestamps']['End Date']
                
                print(f"Q: {q}")
                print(f"   [{cat} | {data['Market Core']['Status']}]")
                
                # Show score only if it exists (Sports)
                if "Score" in data["Context"]:
                    print(f"   Score: {data['Context']['Score']}")
                    
                print(f"   Volume: ${vol:,.2f} | Ends: {end}")
                print("   Prices:")
                for name, price in data['Outcomes'].items():
                    print(f"     - {name}: {price}")
                
                print("-" * 60) # Separator

Processing 19 markets...

Q: Will Al Ittihad Saudi Club win on 2026-01-16?
   [Saudi Professional League | Closed]
   Score: 0-1
   Volume: $97,470.41 | Ends: 2026-01-16 17:30 UTC
   Prices:
     - Yes: 0.0
     - No: 1.0
------------------------------------------------------------
Q: ASVEL Lyon-Villeurbanne vs. Hapoel Tel Aviv
   [Euroleague Basketball | Closed]
   Volume: $29,564.73 | Ends: 2026-01-16 19:00 UTC
   Prices:
     - ASVEL Lyon-Villeurbanne: 0.0
     - Hapoel Tel Aviv: 1.0
------------------------------------------------------------
Q: Khamenei out as Supreme Leader of Iran by January 31?
   [Khamenei out | Active]
   Volume: $35,404,870.19 | Ends: 2026-01-31 00:00 UTC
   Prices:
     - Yes: 0.075
     - No: 0.925
------------------------------------------------------------
Q: Clippers vs. Raptors
   [NBA 2026 | Closed]
   Score: 121-117
   Volume: $2,707,163.12 | Ends: 2026-01-17 00:30 UTC
   Prices:
     - Clippers: 1.0
     - Raptors: 0.0
------------------------------